In [3]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from math import sqrt


In [4]:
df = pd.read_csv("./spam.csv", encoding='latin-1')
clean = df.drop(['Unnamed: 2'], axis=1)
clean = clean.drop(['Unnamed: 3'], axis = 1)
clean = clean.drop(['Unnamed: 4'], axis = 1)
clean.rename(columns = {'v1' : 'Category', 'v2' : 'Message'}, inplace=True)
clean["Message"] = clean["Message"].str.replace('[^a-zA-Z ]','', regex = True)
clean["Message"] = clean["Message"].str.lower()

In [5]:

train_data, test_data = train_test_split(clean, test_size=0.25, random_state=21)

In [6]:
def word_count(message, word_map):
    for word in message.split():
        if word in word_map:
            word_map[word] += 1
        else:
            word_map[word] = 1
    return word_map


In [7]:
def euclidean_difference(train_message_dict, test_message_dict):
    total = 0
    for word in test_message_dict:
        if word in test_message_dict and word in train_message_dict:
            total += (test_message_dict[word]- train_message_dict[word]) ** 2
            del train_message_dict[word]
        else:
            total += test_message_dict[word] ** 2
    for word in train_message_dict:
        total += train_message_dict[word] ** 2
    return sqrt(total)

In [8]:
def spam_or_ham(Kvalues):
    spam_counter, ham_counter = 0, 0
    for item in Kvalues:
        if item[0] == "spam":
            spam_counter += 1
        elif item[0] == "ham":
            ham_counter += 1
    if spam_counter > ham_counter:
        return "spam"
    elif spam_counter < ham_counter:
        return "ham"
    else:
        return "neither"

In [ ]:
def K_nearest_spam_test(train_data, test_messsage, K, train_list):
    sim = []
    test_count = word_count(test_messsage)
    for index in range(len(train_data)):
        diff = euclidean_difference(train_list[index], test_count)
        sim.append([train_data['Category'].values[index], diff])
    sim = sorted(sim, key = lambda x: x[1])
    K_nearest = []
    for i in range(K):
        K_nearest.append(sim[i])
    return spam_or_ham(K_nearest)

In [13]:
def KNN(train_data, test_data, K):
    res = []

    word_map = {}
    train_list = [word_count(x, word_map) for x in train_data['Message']]
    res = [K_nearest_spam_test(train_data, x, K, train_list) for x in train_data['Message']]
    for test_message in test_data:
        sim = []
        test_count = word_count(test_message)
        for index in range(len(train_data)):
            diff = euclidean_difference(train_list[index], test_count)
            sim.append([train_data['Category'].values[index], diff])
        sim = sorted(sim, key = lambda x: x[1])
        K_nearest = []
        for i in range(K):
            K_nearest.append(sim[i])
        res.append(spam_or_ham(K_nearest))
    return res


In [14]:
K = 11
result = KNN(train_data, test_data[:size], K) 
